In [31]:
import psycopg2

In [32]:
create_sql = '''
drop table if exists test.service_windows;
create table if not exists test.service_windows (
    id int, branch_id int, location_id int,
    service_window json
);'''
select_sql = '''
select * from test.service_windows
'''
insert_sql = '''
insert into test.service_windows (id, branch_id, location_id, service_window)
values (1,1,1,
'[
{"sd": "M", "sw": ["0600-1200","1400-1500"]},
{"sd": "T", "sw": ["0600-1200","1400-1500"]}
]');
insert into test.service_windows (id, branch_id, location_id, service_window)
values (2,2,2,
'[
{"sd": "W", "sw": ["0600-1200","1400-1500"]},
{"sd": "H", "sw": ["0600-1200"]}
]');
'''

In [33]:
with psycopg2.connect("dbname=brinks") as conn:
    with conn.cursor() as cur:
        cur.execute(create_sql)
        # conn.commit()
        cur.execute(insert_sql)
        cur.execute(select_sql)
        rec = cur.fetchone()
        print(type(rec[3][0]))
rec


<class 'dict'>


(1,
 1,
 1,
 [{'sd': 'M', 'sw': ['0600-1200', '1400-1500']},
  {'sd': 'T', 'sw': ['0600-1200', '1400-1500']}])